#### Scraping Toronto postal code in wiki, and combine it with geolocation data

In [103]:
!pip install geocoder
!pip install cssselect
import pandas as pd
import requests
import lxml.html as lh

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbor'])
i=0
#For each row, store each first element (header) and an empty list
j = 0
for a in range(0,20) :
  for t in tr_elements[a]:
      i+=1
      name=t.text_content().replace('\n','').replace('/',',')      
      if 'Not' not in name[3:16] :
      
        if name.find('(') < 0 :
          df.loc[j] = [name[0:3], name[3:], name[3:]]
        else:
          df.loc[j] = [name[0:3], name[3:name.find('(')], name[name.find('(')+1:name.find(')')]]        
        j = j + 1

# get geospatial data
df2 = pd.read_csv("http://cocl.us/Geospatial_data")

# Join two data frame
neighborhoods = df.set_index('Postal Code').join(df2.set_index('Postal Code'))


neighborhoods.head()



,Borough,Neighbor,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [102]:
neighborhood.shape

(103, 4)

In [105]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [107]:
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbor']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Toronto borough Analysis

In [119]:
toronto_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Borough,Neighbor,Latitude,Longitude
0,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [120]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [121]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbor']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork